In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.linalg as la
import scipy.sparse as sparse

In [2]:
np.set_printoptions(precision=2)

In [3]:
#Create grid

Nx = 20                              #dimensions
Ny = 30

x_I, x_F = 0, np.pi                  #boundary points
y_I, y_F = 0, np.pi

dx = (x_F - x_I)/Nx                  #step sizes
dy = (y_F - y_I)/Ny
step_rat = dx**2/dy**2

X = np.linspace(x_I, x_F, Nx+1)      #grid
Y = np.linspace(y_I, y_F, Ny+1)

x, y = np.meshgrid(X,Y)              #2D grid




In [4]:
#Create penta-diagonal matrix of 2D finite difference discretisation
d1 = step_rat*np.ones((Nx-1)*(Ny-1) - (Nx-1))          #1st lower diagonal
d3 = -2*(step_rat+1)*np.ones((Nx-1)*(Ny-1))       #1st upper diagolan
d5 = step_rat*np.ones((Nx-1)*(Ny-1) - (Nx-1))          #main diagonal

d2 = np.ones((Nx-1)*(Ny-1) - 1)                   #2nd lower diagonal
d4 = np.ones((Nx-1)*(Ny-1) - 1)                   #2nd upper diagonal    
d2[Nx-2:len(d2):Nx-1] = 0
d4[Nx-2:len(d2):Nx-1] = 0

offsets = [-(Nx-1), -1, 0, 1, Nx-1]                       #diagonal positions

M = sparse.diags([d1, d2, d3, d4, d5], offsets).toarray()  #the sparse 5diagonal matrix
# print(M)
# print('')
# print(d2)
# print('')
# print(d4)
# np.nonzero(d2==0)[0]

In [5]:
#Define the source term and the analytical solution
def source(x, y): 

    f = -2*np.sin(x)*np.sin(y) 

    return f

def u_analytical(x, y):

    u_an = np.sin(x)*np.sin(y) + 2*(x-np.pi)*y + 1 

    return u_an

def bound(x, y):

    b = np.zeros([(Nx-1)*(Ny-1)])
    b[0:Nx-1] -= step_rat*u_analytical(x, y)[0, 1:Nx]
    b[-(Nx-1):] -= step_rat*u_analytical(x, y)[-1, 1:Nx]
    b[0::Nx-1] -= u_analytical(x, y)[1:Ny, 0]
    b[Nx-2::Nx-1] -= u_analytical(x, y)[1:Ny, -1]
    return b


In [6]:
#Test example
u_an = u_analytical(x, y)                                                    #analytical solution for comparison
# print(u_an)
rhs = (dx)**2*source(x, y)[1:Ny, 1:Nx].reshape((Nx-1)*(Ny-1)) + bound(x,y)    #create right hand side

# for j in range(Ny):
#     rhs[0, j] -= u_an[0, j] 

# print(rhs)

u_num = la.solve(M, rhs).reshape(Ny-1, Nx-1)                      #solve for the internal grid points
u_num = np.insert(u_num, [0, Ny-1], [u_an[0,1:Nx], u_an[-1,1:Nx]], axis = 0)                  #add the boundary values
# u_num = np.insert(u_num, [0, Nx-1], [u_an[1:Ny,0], u_an[1:Ny,-1]], axis = 1)

u_num = np.hstack((u_num, u_an[:,Nx].reshape(Ny+1, 1)))
u_num = np.hstack((u_an[:,0].reshape(Ny+1, 1), u_num))
# print(u_num)
                                        

# print(u_an)
# print("")
# print(u_num)

In [7]:
#Plot the results
#magic command for figure in external figure
# %matplotlib qt                                


fig = plt.figure()                                #create 3D figure and axes
ax = fig.add_subplot(111, projection='3d')

scat = ax.scatter(x, y, u_num, c = 'r')                  #plot the numerical solution vs the analytical
surf = ax.plot_surface(x, y, u_an)

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel($\sinx\siny+2(x-\pi)y+1$)

surf._facecolors2d=surf._facecolors3d
surf._edgecolors2d=surf._edgecolors3d

ax.legend()

plt.show()